In [2]:
import os
import pandas as pd
import json
from io import StringIO



data_dir = './'
if not os.path.exists(data_dir):
    print(f"Error: Data directory '{data_dir}' not found.")
    
standardized_data = []
# These are the standard names we will use for the DataFrame columns.
standard_columns = ['timestamp', 'co2', 'humidity', 'temperature']

for filename in os.listdir(data_dir):
    if filename.endswith('.ndjson'):
        # Create a clean room name, e.g., 'room_a.txt' -> 'Room A'
        room_name = filename.split('.')[0].replace('_', ' ').title()
        file_path = os.path.join(data_dir, filename)
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for i, line in enumerate(f):
                    # Skip empty or whitespace-only lines
                    if not line.strip():
                        continue
                    
                    try:
                        # Load the JSON object from the line
                        data_dict = json.loads(line)
                        
                        # Get values in their insertion order (works in Python 3.7+)
                        values = list(data_dict.values())
                        
                        # Validate that we have the expected number of fields
                        if len(values) != 4:
                            print(f"Warning: Skipping malformed line {i+1} in {filename} (expected 4 fields, got {len(values)}).")
                            continue

                        # Create a standardized dictionary by zipping keys and values
                        record = dict(zip(standard_columns, values))
                        record['room'] = room_name  # Add the room name
                        standardized_data.append(record)
                        
                    except (json.JSONDecodeError, IndexError) as e:
                        print(f"Warning: Skipping corrupted line {i+1} in {filename}. Error: {e}")
                        continue

        except Exception as e:
            print(f"Error reading file {filename}: {e}")
            continue
            
if not standardized_data:
    print("Error: No data could be loaded. Check file contents and paths.")

# Create the DataFrame from our list of standardized records
df = pd.DataFrame(standardized_data)

# Convert relevant columns to the correct data types, coercing errors
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
for col in ['co2', 'humidity', 'temperature']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop any rows where critical data conversion failed
df.dropna(subset=['timestamp', 'co2', 'humidity', 'temperature'], inplace=True)



In [3]:
df.tail(30)

,timestamp,co2,humidity,temperature,room
5350,2025-07-17 06:06:58.190029+00:00,688.06,37.44,23.55,Sensor Data Room 4
5351,2025-07-17 06:21:58.190029+00:00,557.35,41.41,23.09,Sensor Data Room 4
5352,2025-07-17 06:36:58.190029+00:00,486.17,45.77,23.74,Sensor Data Room 4
5353,2025-07-17 06:51:58.190029+00:00,605.77,49.93,22.20,Sensor Data Room 4
5354,2025-07-17 07:06:58.190029+00:00,557.86,50.58,20.16,Sensor Data Room 4
5355,2025-07-17 07:21:58.190029+00:00,530.83,39.04,23.91,Sensor Data Room 4
5356,2025-07-17 07:36:58.190029+00:00,720.09,38.41,22.49,Sensor Data Room 4
5357,2025-07-17 07:51:58.190029+00:00,600.75,46.84,22.47,Sensor Data Room 4
5358,2025-07-17 08:06:58.190029+00:00,482.75,47.17,23.72,Sensor Data Room 4
5359,2025-07-17 08:21:58.190029+00:00,684.30,32.12,21.53,Sensor Data Room 4
